In [1]:
# -*- coding: utf-8 -*-
import sys, os, shutil, re, json, string, random, math, unittest, glob, time
import numpy as np
# zip(),zip(*),[ for in if else],reduce(),iter()
# execfile(),eval()
# import imp,__import__(),globals() , locals()

# re.match(pattern, string,re.I|re.U)
# re.search(pattern, string,re.I|re.U)
# re.findall(string[, pos[, endpos]])
# re.finditer(pattern, string, flags=0)
# re.sub(pattern, repl, string, count=0, flags=0)
# re.split(pattern, string[, maxsplit=0, flags=0])
# re.compile(pattern[, flags])
# matchObj = re.match( r'(.*) are (.*?) .*', line, re.M|re.I)
# matchObj.group()，matchObj.group(1)，matchObj.group(2)
# obj //none
# patter=re.compile(pattern,re.I|re.U)


# filter(function, iterable)
# filter(lambda n:n % 2 == 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# iter(object[, sentinel])
# for i in iter([1, 2, 3]):
# reduce(function, iterable[, initializer])
# reduce(lambda x, y: x+y, [1,2,3,4,5])
reduce(lambda x, y: x.union(set([y])), [set()]+[1,2,3,4,5])

# time
# t1=time.process_time()
# t2=time.process_time()
# t2-t1


# import six


# https://www.cnblogs.com/rainfd/p/slots.html
# slots

import arcpy

# http://resources.arcgis.com/zh-cn/help/main/10.2/index.htm
# http://desktop.arcgis.com/zh-cn/
# http://server.arcgis.com/zh-cn/

# http://muchong.com/t-11204040-1
# https://www.epubit.com/bookDetails?id=N20489

oldout,olderr=sys.stdout,sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout,sys.stderr=oldout,olderr
arcpy.env.outputMFlag=u'Disabled'
arcpy.env.outputZFlag=u'Disabled'
def my_point_hash(point):
    return hash(point.X) ^ hash(point.Y) ^ hash(point.Z) ^ hash(point.M)
arcpy.Point.__eq__=arcpy.Point.equals
arcpy.Point.__hash__=my_point_hash
def my_geometry_hash(geometry):
    ret_hash=hash(geometry.area)
    try:
        ret_hash= ret_hash ^ hash(geometry.centroid)
    except:
        pass# 空几何,没有点，或者多部分
    return ret_hash
# 有考虑保留有效位数的hash版本，为了和arcpy.Geometry.equals配合。'%.6E' % 4654644644.14542145,,'{:.6e}'.format(12154.14542145)
arcpy.Polygon.__eq__=arcpy.Polygon.equals
arcpy.Geometry.__eq__=arcpy.Geometry.equals
arcpy.PointGeometry.__eq__=arcpy.PointGeometry.equals
arcpy.Geometry.__hash__=arcpy.Polygon.__hash__=arcpy.PointGeometry.__hash__=my_geometry_hash
arcpy.Extent.__eq__=arcpy.Extent.equals
try:
    if convexHull_flag:
        pass
except :
    def polygon_refresh(polygon):
        return arcpy.Polygon(arcpy.Array([part for part in polygon ]))
    old_convexHull_handle=arcpy.Polygon.convexHull
    def MyconvexHull(polygon):
        return polygon_refresh(old_convexHull_handle(polygon))
    arcpy.Geometry.convexHull=arcpy.Polygon.convexHull=arcpy.Polyline.convexHull=arcpy.PointGeometry.convexHull=MyconvexHull
    convexHull_flag=True
arcpy.env.workspace='in_memory'


class OverWriteFunction(object):
    def __init__(self,old_func):
        self.superFunc=old_func
    def __str__(self):
        return '<%s %s>'%('OverWriteFunctionSaveStdoutStderr',self.super.func_name)
    def func_name(self):
        return self.__str__()
    def __call__(self,*a,**k):
        self.superFunc(*a,**k)
        # raise Exception('OverWriteFunction.__call__ null')
        pass
    pass

class OverWriteFunctionSaveStdoutStderr(OverWriteFunction):
    def __call__(self,*a,**k):
        oldout,olderr=sys.stdout,sys.stderr
        self.superFunc(*a,**k)
        sys.stdout,sys.stderr=oldout,olderr
        pass
    pass

if str(type(sys.stdout))=='<class \'ipykernel.iostream.OutStream\'>':
    # 为了和jupyter兼容的代码
    # AttributeError: 'geoprocessing sys.stdout object' object has no attribute 'flush'
    if arcpy.CalculateField_management.func_name == 'CalculateField':
        arcpy.CalculateField_management=OverWriteFunctionSaveStdoutStderr(arcpy.CalculateField_management)
        arcpy.management.CalculateField=OverWriteFunctionSaveStdoutStderr(arcpy.management.CalculateField)
    if arcpy.FeatureClassToGeodatabase_conversion.func_name == 'FeatureClassToGeodatabase':
        arcpy.FeatureClassToGeodatabase_conversion=OverWriteFunctionSaveStdoutStderr(arcpy.FeatureClassToGeodatabase_conversion)
        arcpy.conversion.FeatureClassToGeodatabase=OverWriteFunctionSaveStdoutStderr(arcpy.conversion.FeatureClassToGeodatabase)



In [2]:
ghyt=r'D:\gongzuo\temp\局调\2020-国庆\更新成果shp0619带40\GHYT.shp'
in_fc=r'D:\gongzuo\temp\局调\2020-国庆\建德复垦局调项目范围\复垦项目范围_GHYTGZQ.shp'

In [3]:


class IncrementalNumberByKey(object):
    def __init__(self,start=1):
        self.map={}
        self.start=1
        pass
    def get_num(self,key):
        num=self.map.setdefault(key,self.start-1)
        num+=1
        self.map[key]=num
        return num
    def set_num(self,key,num):
        num=max(self.map[key],num)
        self.map[key]=num
    pass



In [4]:
arcpy.CopyFeatures_management(ghyt,'ghyt')

<Result 'in_memory\\ghyt'>

In [6]:
[field.name for field in arcpy.ListFields('ghyt')]

[u'FID',
 u'Shape',
 u'OBJECTID',
 u'YSDM',
 u'XZQDM',
 u'XZQMC',
 u'YTDKBH',
 u'GHYTDM',
 u'GHYTBM',
 u'GHYTMC',
 u'MJ',
 u'JMJ',
 u'BZ',
 u'JNLX',
 u'PZWH',
 u'HQRQ',
 u'XZNTLX',
 u'SHAPE_Leng']

In [5]:
arcpy.CopyFeatures_management(in_fc,'xmfw')

<Result 'in_memory\\xmfw'>

In [16]:
list_[[row.getValue('FID'),row.getValue('XZQDM'),row.getValue('GHYTBM'),row.getValue('YTDKBH')] 
 for row in arcpy.SearchCursor('ghyt') 
 if row.getValue('GHYTBM').strip() != '']



293 330182001001 A2 A2-001


In [19]:
del scur

In [26]:
f1=r'C:\Users\HP\Documents\ArcGIS\复垦项目范围_GHYTGZQ_GHYT01_nd4.shp'
f2=r'C:\Users\HP\Documents\ArcGIS\复垦项目范围_GHYTGZQ_GHYT01_s2.shp'

In [34]:
f1=r'D:\gongzuo\temp\局调\2020-国庆\建德复垦局调项目范围\复垦项目范围_GHYTGZQ_GHYT011.shp'
# f1=r'C:\Users\HP\Documents\ArcGIS\复垦项目范围_GHYTGZQ_GHYT01_nd4.shp'
f2=r'C:\Users\HP\Documents\ArcGIS\复垦项目范围_GHYTGZQ_GHYT01_s2.shp'
scur1=arcpy.SearchCursor(f1)
scur2=arcpy.SearchCursor(f2)
i=1
while True:
    row1=scur1.next()
    row2=scur2.next()
    if row1 is None:
        break
    FID1,XZQDM1,YYTDKBH1,YTDKBH1=row1.getValue('FID'),row1.getValue('XZQDM'),row1.getValue('YYTDKBH'),row1.getValue('YTDKBH')
    FID2,XZQDM2,YYTDKBH2,YTDKBH2=row2.getValue('FID'),row2.getValue('XZQDM'),row2.getValue('YYTDKBH'),row2.getValue('YTDKBH')
    if (FID1!=FID2) or (XZQDM1!=XZQDM2) or (YYTDKBH1!=YYTDKBH2) or (YTDKBH1!=YTDKBH2) :
        print FID1,FID2
        print YYTDKBH1,YTDKBH1,YYTDKBH2,YTDKBH2
#     if i % 10 ==0:
#         print i
    i+=1




159 159
D2-009 B2-068 D2-016 B2-068
161 161
D1-001 B2-064 D1-002 B2-064
187 187
D6-001 B2-063 D6-002 B2-063
191 191
D6-001 B2-064 D6-003 B2-064
261 261
D6-002 B2-035 D6-003 B2-035
300 300
农-002 B2-043 农-001 B2-043
301 301
农-003 B2-045 农-002 B2-045
317 317
建-008 B2-014 建-007 B2-014
334 334
建-007 B2-013 建-006 B2-013
377 377
D6-008 B2-147 D6-015 B2-147
394 394
D6-005 B2-137 D6-013 B2-137
521 521
农-004 B2-049 农-003 B2-049
522 522
农-005 B2-049 农-003 B2-049
525 525
农-003 B2-047 农-002 B2-047
566 566
D2-002 B2-017 D2-004 B2-017
593 593
建-004 B2-034 建-003 B2-034
600 600
建-004 B2-160 建-002 B2-160
601 601
建-009 B2-162 建-004 B2-162
604 604
建-011 B2-164 建-006 B2-164
605 605
建-010 B2-163 建-005 B2-163
607 607
建-008 B2-161 建-003 B2-161
609 609
建-002 B2-038 建-001 B2-038
610 610
建-003 B2-033 建-002 B2-033
612 612
建-002 B2-032 建-001 B2-032
613 613
建-003 B2-065 建-002 B2-065
658 658
建-005 B2-081 建-003 B2-081
664 664
D2-010 B2-105 D2-014 B2-105
667 667
D6-013 B2-040 D6-014 B2-040
714 714
D6-008 B2-145 D6-014

In [36]:

fc=r'D:\gongzuo\temp\局调\2020-国庆\复垦项目范围\复垦项目范围.shp'
scur=arcpy.SearchCursor(fc)
list_row=[[row.getValue('shape'),row.getValue('XH'),row.getValue('BZ')]for row in scur]
del row 
del scur

In [38]:
map_row_by_XH={}
for row_id,row in enumerate(list_row):
    shape,XH,BZ=row
    map_row_by_XH.setdefault(XH,[]).append(row_id)

In [39]:
map_row_by_XH

{1: [0],
 2: [1, 2],
 5: [3],
 6: [4, 5, 6],
 9: [7, 8],
 10: [9],
 13: [10],
 17: [11, 12, 13, 14, 15],
 19: [16, 17, 18, 19, 20, 21, 22],
 20: [23, 24, 25, 26, 27],
 21: [28, 29, 30, 31],
 22: [32],
 23: [33, 34, 35, 36],
 24: [37, 38, 39, 40, 41],
 25: [42],
 26: [43, 44, 45, 46, 47, 48, 49, 50, 51],
 27: [52, 53, 54, 55],
 28: [56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75],
 29: [76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94],
 30: [95, 96, 97, 98, 99, 100, 101, 102],
 31: [103],
 33: [104, 105],
 36: [106, 107, 108],
 37: [109],
 38: [110, 111],
 39: [112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128],
 41: [129],
 42: [130, 131, 132, 133, 134, 135, 136, 137, 138],
 43: [139, 140],
 44: [141, 142, 143, 144, 145, 146, 147],
 45: [148, 149],
 46: [150, 151, 152, 153],
 47: [154, 155,

In [40]:
for key,value in map_row_by_XH.items():
    if (value)>1:
        print key

1
2
5
6
9
10
13
17
19
20
21
22
23
24
25
26
27
28
29
30
31
33
36
37
38
39
41
42
43
44
45
46
47
48
49
53
54
56
58
59
61
63
64
65
67
69
70
71
72
73
74
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
122
123
125
126
127
128
129
130
131
132
134
135
136
137
138
139
140


In [45]:

fc=r'D:\gongzuo\temp\局调\2020-国庆\复垦项目范围\新建文件夹\复垦项目范围_Dissolve3.shp'
scur=arcpy.SearchCursor(fc)
list_row=[[row.getValue('shape'),row.getValue('XH')]for row in scur]
del row 
del scur

map_row_by_XH2={}
for row_id,row in enumerate(list_row):
    shape,XH=row
    map_row_by_XH2.setdefault(XH,[]).append(row_id)

{1: [0],
 2: [1, 2],
 5: [3],
 6: [4, 5, 6],
 9: [7, 8],
 10: [9],
 13: [10],
 17: [11, 12, 13, 14, 15],
 19: [16, 17, 18, 19, 20, 21, 22],
 20: [23, 24, 25, 26, 27],
 21: [28, 29, 30, 31],
 22: [32],
 23: [33, 34, 35, 36],
 24: [37, 38, 39, 40, 41],
 25: [42],
 26: [43, 44, 45, 46, 47, 48, 49, 50, 51],
 27: [52, 53, 54, 55],
 28: [56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75],
 29: [76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94],
 30: [95, 96, 97, 98, 99, 100, 101, 102],
 31: [103],
 33: [104, 105],
 36: [106, 107, 108],
 37: [109],
 38: [110, 111],
 39: [112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128],
 41: [129],
 42: [130, 131, 132, 133, 134, 135, 136, 137, 138],
 43: [139, 140],
 44: [141, 142, 143, 144, 145, 146, 147],
 45: [148, 149],
 46: [150, 151, 152, 153],
 47: [154, 155,

In [48]:
for key,value in map_row_by_XH.items():
    if (value)>1:
        value2=map_row_by_XH2[key]
        if len(value2)!=len(value):
            print key

49
140


In [55]:
def deal(bz):
    bz=bz.encode('utf-8')
    this_str="建德市"
    strlen=len(this_str)
    if bz[:strlen]==this_str:
        return bz[strlen:].decode('utf-8')
    return bz.decode('utf-8')

print deal('建德市下涯镇和睦新型墙体材料厂（砖瓦窑）建设用地复垦项目拆旧地块2')

建德市
下涯镇和睦新型墙体材料厂（砖瓦窑）建设用地复垦项目拆旧地块2


In [57]:
"建德市".decode('utf-8')

u'\u5efa\u5fb7\u5e02'